In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import folium
from folium import Choropleth
from folium import GeoJson
import geopandas as gpd

import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [ ]:
pd.set_option('display.max_columns', None)

df1 = pd.read_csv('../bin/data1.csv')
df2 = pd.read_csv('../bin/data2.csv')
df3 = pd.read_csv('../bin/data3.csv')

df = pd.concat([df1, df2, df3], ignore_index=True)

df.head()

## Limpieza preliminar

In [ ]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

In [ ]:
df['tipo_cambio'].unique()

#Hay nans que se toman como floats. Lo convierto a "no disponible"
df['tipo_cambio'] = df['tipo_cambio'].fillna("no disponible")

#No sé qué es manual automatizada, si queremos que sea manual hay que poner la lambda al revés
df['tipo_cambio'] = df['tipo_cambio'].apply(lambda x: 'Manual' if 'Manual' in x else 'Automático')

In [ ]:
df["carroceria"] = df["carroceria"].str.replace("Carrocería","")

df["combustible"] = df["combustible"].str.replace("combustible ","").str.replace("Combustible ","")

In [ ]:
df['fecha_matriculacion'] = df['fecha_matriculacion'].str.split("Matriculado: ").str[1]

def limpiar_fecha(fecha):
    try:
        mes, ano = fecha.split('/')
        return int(mes), int(ano)
    except:
        return np.nan, int(fecha)

df[['mes_matriculacion', 'ano_matriculacion']] = df['fecha_matriculacion'].apply(lambda x: pd.Series(limpiar_fecha(x)))

df = df.drop(columns=['fecha_matriculacion'])

In [ ]:
df['mes_matriculacion'] = df['mes_matriculacion'].fillna(0).astype('int64')
df['ano_matriculacion'] = df['ano_matriculacion'].fillna(0).astype('int64')

In [ ]:
df = df[df["tipo_vendedor"] == "Profesional"]
df = df.drop(columns=['tipo_vendedor'])

In [ ]:
df["garantia"] = df["garantia"].str.split(' ').str[0].replace("Sí",np.nan).replace("No",0).astype(float)

In [ ]:
df["precio_contado"] = pd.to_numeric(df["precio_contado"], errors='coerce')

df["precio_nuevo"] = pd.to_numeric(df["precio_nuevo"], errors='coerce')

df['largo'] = pd.to_numeric(df['largo'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['ancho'] = pd.to_numeric(df['ancho'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['alto'] = pd.to_numeric(df['alto'].str.extract('(\d+\.?\d*)')[0], errors='coerce')

df['capacidad_maletero'] = pd.to_numeric(df['capacidad_maletero'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')
df["num_plazas"] = df["num_plazas"].str.split("s").str[1].astype("Int64")
df["batalla"] = df["batalla"].str.replace('Batalla (mm)\r\n', '').str.replace('.','').astype("Int64")
df["peso"] = df["peso"].str.replace('Peso/Masa max. autorizado (kg)', '').str.replace('.','').astype("Int64")
df["num_puertas"] = df["num_puertas"].str.extract(r'(\d+)').astype("Int64")
df["consumo_medio"] = df["consumo_medio"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_carretera"] = df["consumo_carretera"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_urbano"] = df["consumo_urbano"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["co2"] = df["CO2"].str.extract(r'(\d{1,3})(?= g/km)')[0].astype(float)
df["num_cilindros"] = df["cilindros"].str.extract(r'(\d+)').astype("Int64")
df["cilindrada"] = df["cilindrada"].str.extract(r'(\d+,\d+|\d+\.\d+)', expand=False).str.replace(',', '').str.replace('.', '').astype("Int64")
df["deposito"] = df["deposito"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

df["num_marchas"] = df["num_marchas"].str.extract(r'(\d+)').astype("Int64")
df["potencia_kw"] = df["potencia_kw"].str.extract(r'(\d+)').astype("Int64")
df["potencia_cv"] = df["potencia_cv"].str.extract(r'(\d+)').astype("Int64")
df["par"] = df["par"].str.extract(r'(\d+)').astype("Int64")
df["velocidad_max"] = df["velocidad_max"].str.extract(r'(\d+)').astype("Int64")
df["aceleracion"] = df["aceleracion"].str.extract(r'(\d+,\d+|\d+\.\d+)')[0].str.replace(",", ".").astype(float)
df['id_sobrealimentacion'] = df['sobrealimentacion'].apply(lambda x: x.replace('Sobrealimentación', '').replace('Tipo de sobrealimentador', 'no disponible').strip() if pd.notna(x) and x != '-' else 'no disponible')

In [ ]:
df['id_sobrealimentacion'].unique()

In [ ]:
df["traccion"].unique()

df["traccion"] = df['traccion'].fillna("no disponible")
df["traccion"] = df["traccion"].str.replace('Tracción', '').str.replace('Todo terreno tracción', '').apply(lambda x: x.strip())

In [ ]:
marca_listado = pd.read_csv('../bin/listado_marcas.csv')

lista_marcas = [marca.upper() for marca in marca_listado['nombre_marca'].tolist()]

df['marca_modelo'] = df['marca_modelo'].fillna('').str.replace("Detalles ", "").str.strip().str.upper()

df['marca'] = df['marca_modelo'].apply(lambda x: next((marca for marca in lista_marcas if marca in x), np.nan))
df['modelo'] = df.apply(lambda x: re.sub(r'\s*\(.*?\)', '', x['marca_modelo'].replace(x['marca'], "").strip()).strip() if pd.notnull(x['marca']) else x['marca_modelo'], axis=1)


In [ ]:
df = df.drop(columns=['marca_modelo', 'CO2', 'cilindros', 'sobrealimentacion', 'carroceria'])

In [ ]:
df.rename(columns={'distintivo_ambiental': 'id_distintivo_ambiental'
                  , 'marca': 'id_marca'
                  , 'modelo': 'id_modelo'
                  , 'nombre_vendedor': 'id_concesionario'
                  , 'provincia': 'id_provincia'
                  , 'traccion': 'id_traccion'}, inplace=True)

In [ ]:
df['combustible'] = (
    df['combustible']
    .str.replace('-', 'no disponible')
    .str.replace('Diésel', 'Diesel')
    .str.replace('Gasolina y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Diesel y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Corriente eléctrica', 'Eléctrico', regex=False)
    .str.replace(r'\bHíbrido\b(?!.*Enchufable)', 'Híbrido Enchufable', regex=True)
)

In [ ]:
df['id_concesionario'] = df['id_concesionario'].apply(lambda x: x.split('-')[0])

In [ ]:
for columna in df.columns:
    df[columna] = df[columna].replace(["no disponible", "<NA>", "-", "0", 0, np.nan], None)
    

In [ ]:
df = df[df['precio_contado'].notna()]

## Rtdo limpieza

In [ ]:
df.sample(10)

In [ ]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

In [ ]:
df.to_csv('../bin/data_preprocess.csv', index=False)

## Gráficos

In [ ]:
plt.figure(figsize=(10, 6))
for tipo_combustible in df['combustible'].unique():
    subset = df[df['combustible'] == tipo_combustible]
    plt.scatter(subset['potencia_cv'], subset['precio_contado'], label=tipo_combustible, alpha=0.6)

# Configurar etiquetas y título
plt.xlabel('Potencia (cv)')
plt.ylabel('Precio')
plt.title('Relación entre Precio y Potencia por Tipo de Combustible')
plt.legend(title='Combustible')
plt.yscale('log')

#Guardamos la imagen para usarla en Streamlit
plt.savefig('../bin/imagenes/grafico_precio_potencia.png')

plt.show()



In [ ]:
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df['kilometraje'], df['precio_contado'], c=df['ano_matriculacion'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Año de Matriculación')

# Configurar etiquetas y título
plt.xlabel('Kilometraje')
plt.ylabel('Precio Contado')
plt.title('Relación entre Kilometraje y Precio Contado por Año de Matriculación')
plt.yscale('log')

#Guardamos la imagen para usarla en Streamlit
plt.savefig('../bin/imagenes/grafico_kilometraje_precio.png')

plt.show()



In [ ]:
precio_medio = df.groupby('id_marca')[['precio_nuevo', 'precio_contado']].mean().sort_values(by='precio_contado', ascending=False)

precio_medio.plot(kind='bar', figsize=(18, 7))
plt.xlabel('Marca')
plt.ylabel('Precio Medio')
plt.title('Comparación del Precio Nuevo y Precio Contado por Marca')
plt.xticks(rotation=45, fontsize=8)

#Guardamos la imagen para usarla en Streamlit
plt.savefig('../bin/imagenes/grafico_precio_nuevo_contado.png')

plt.show()



In [ ]:
plt.figure(figsize=(20, 15))
df.boxplot(column='precio_contado', by='id_distintivo_ambiental', grid=False)
plt.xlabel('Distintivo Ambiental')
plt.ylabel('Precio Contado')
plt.title('Distribución del Precio Contado por Distintivo Ambiental')
plt.suptitle('')  # Elimina el título automático generado por boxplot
plt.yscale('log')

#Guardamos la imagen para usarla en Streamlit
plt.savefig('../bin/imagenes/grafico_precio_distintivo.png')

plt.show()



## Visualizaciones

In [ ]:
df.columns

In [ ]:
categorical_columns = ['id_provincia', 'id_concesionario', 'id_distintivo_ambiental','combustible','tipo_cambio', 'id_traccion', 'mes_matriculacion'
               , 'ano_matriculacion','id_sobrealimentacion', 'id_marca', 'id_modelo']
continuous_columns = []

for columna in df.columns: 
    if columna not in categorical_columns: 
        continuous_columns.append(columna)

In [ ]:
def metricas (df, columna):
    
    if columna in categorical_columns:
        
        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 2, 1)
        sns.countplot(data=df, x=columna)
        plt.title(f'Distribución de {columna}')
        plt.xticks(rotation=45, ha='right')
        
        plt.subplot(1, 2, 2)
        sns.boxplot(data=df, x=columna, y='precio_contado')
        plt.title(f'Boxplot de {columna} con precio')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')
        plt.show()
    
    elif columna in continuous_columns:

        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 3, 1)
        sns.histplot(df[columna], kde=True)
        plt.title(f'Histograma de {columna}')
        
        
        plt.subplot(1, 3, 2)
        sns.boxplot(x=df[columna])
        plt.title(f'Boxplot de {columna}')
        plt.xticks(rotation=45, ha='right')
        
        plt.subplot(1, 3, 3)
        sns.scatterplot(x=df[columna], y=df['precio_contado'])
        plt.title(f'Scatterplot de {columna} con precio')
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')
        plt.show()

In [ ]:
for columna in continuous_columns:
    metricas(df, columna)

In [ ]:
for columna in categorical_columns:
    metricas(df, columna)

In [ ]:
correlation_matrix = df[continuous_columns].corr()

plt.figure(figsize=(25, 15))

sns.heatmap(correlation_matrix, annot=True)

plt.title('Mapa de Calor de Correlaciones')
plt.show()